## Preparation

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from IPython.display import SVG

Using TensorFlow backend.
/home/emma/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Set parameters

In [2]:
p = 4

Define the model architecture

In [3]:
def build_model(p, n):
    model = Sequential()

    model.add(LSTM(units=n, activation='sigmoid', input_dim=p, dropout=0.5, recurrent_dropout=0.5))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='mae',
                  optimizer='adam',
                  metrics=['mse'])
    
    return model

Load the data

In [4]:
stocks = {
    'ndx': pd.read_csv('^NDX_daily.csv'),
    'dji': pd.read_csv('^DJI_daily.csv'),
    'gspc': pd.read_csv('^GSPC_daily.csv')
}

Define an access structure for the data

In [5]:
class Data:
    def scale(self, y):
            return (y-self.initial_range[0])/(self.initial_range[1]-self.initial_range[0])/2 + 0.25
    
    def unscale(self, y):
            return (y - 0.25) * 2 * (self.initial_range[1]-self.initial_range[0]) + self.initial_range[0]
        
    def __init__(self, stock, split):
        input = np.asarray(stock['Close'][1:])

        input = (input[1:]-input[:-1])/input[:-1] # Single day returns (Used in previous research)
        # input = (input[1:]-input[0])/input[0] # Returns since start of data (Alternative)

        input = TimeseriesGenerator(input, input, p+1)[0]

        X_with_ref = np.asarray(input[0])
        y = np.asarray(input[1])

        X = np.empty([len(X_with_ref), len(X_with_ref[0])-1])
        for i in range(len(y)):
            y[i] = y[i]-X_with_ref[i][0]
            X[i] = X_with_ref[i][1:]-X_with_ref[i][0]

        # Reshape X into single timestep
        X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

        self.initial_range = (min(y), max(y))

        self.X = X
        self.y = y
        
        # Normalize y into range [0.25, 0.75]
        self.y_scaled = self.scale(y)

        self.X_train = self.X[:int(split*len(y))]
        self.y_train = self.y[:int(split*len(y))]
        self.y_train_scaled = self.y_scaled[:int(split*len(y))]
        self.X_test = self.X[int(split*len(y)):]
        self.y_test = self.y[int(split*len(y)):]
        self.y_test_scaled = self.y_scaled[int(split*len(y)):]

Choosing the data for the experiments

In [6]:
data = Data(stocks['ndx'], 0.8)

## Experiment 1: Training overview

In [ ]:
model1 = build_model(p, 1)
model16 = build_model(p, 16)
model32 = build_model(p, 32)

def train(model):
    return model.fit(data.X_train, data.y_train_scaled, validation_data=(data.X_test, data.y_test_scaled), epochs=10000, batch_size=32, verbose=0)

history1 = train(model1)
history16 = train(model16)
history32 = train(model32)

In [66]:
output_notebook()

start = 0

p1 = figure()
p1.line(range(len(history1.history['val_loss'])), history1.history['val_loss'], color="black")
p1.xaxis.axis_label = "Epoch"
p1.yaxis.axis_label = "MAE"
p1.title.text = "n = 1"
show(p1)

p16 = figure()
p16.line(range(len(history16.history['val_loss'])), history16.history['val_loss'], color="black")
p16.xaxis.axis_label = "Epoch"
p16.yaxis.axis_label = "MAE"
p16.title.text = "n = 16"
show(p16)

p32 = figure()
p32.line(range(len(history32.history['val_loss'])), history32.history['val_loss'], color="black")
p32.xaxis.axis_label = "Epoch"
p32.yaxis.axis_label = "MAE"
p32.title.text = "n = 31"
show(p32)

Loading BokehJS ...

## Experiment 2: Fitting speed

In [ ]:
results_traintime = []

for n in range(1,32):
    model = build_model(p, n)
    results_traintime.append(model.fit(data.X_train, data.y_train_scaled, validation_data=(data.X_test, data.y_test_scaled), callbacks=[EarlyStopping(monitor='loss', patience=1000)], epochs=10000, batch_size=32, verbose=0))

In [42]:
output_notebook()

p1 = figure()
p1.line(range(1, len(results_traintime)+1), [len(history.history['val_loss']) for history in results_traintime], color="black")
p1.xaxis.axis_label = "Number of nodes in hidden layer"
p1.yaxis.axis_label = "Number of epochs trained"
show(p1)

Loading BokehJS ...

## Experiment 3: Overfitting speed

In [ ]:
results_overfit = []

for n in range(1,32):
    model = build_model(p, n)
    model.fit(data.X_train, data.y_train_scaled, validation_data=(data.X_test, data.y_test_scaled), epochs=10000, batch_size=32, verbose=0)
    results_overfit.append([n] + model.evaluate(data.X_test, data.y_test_scaled))

In [11]:
output_notebook()

p1 = figure()
p1.line(np.transpose(results_overfit)[0], np.transpose(results_overfit)[1], color="black")
p1.xaxis.axis_label = "Number of nodes in hidden layer"
p1.yaxis.axis_label = "MAE"
show(p1)

Loading BokehJS ...

## Experiment 4: Optimal accuracy

In [ ]:
results_final = []

for n in range(1,32):
    model = build_model(p, n)
    model.fit(data.X_train, data.y_train_scaled, validation_data=(data.X_test, data.y_test_scaled), epochs=100, callbacks=[EarlyStopping(monitor='loss', patience=1000)], batch_size=32, verbose=0)
    results_final.append([n] + model.evaluate(data.X_test, data.y_test_scaled))

In [65]:
output_notebook()

p1 = figure()
p1.line(np.transpose(results_final)[0], np.transpose(results_final)[1], color="black")
p1.xaxis.axis_label = "Number of nodes in hidden layer"
p1.yaxis.axis_label = "MAE"
show(p1)

Loading BokehJS ...